# Efektywne programowanie w języku Python 

## wykład 12

![alt text](images/headlogo.png "headlogo")

Cython is an optimising static compiler for both the Python programming language and the extended Cython programming language (based on Pyrex). It makes writing C extensions for Python as easy as Python itself.

Cython gives you the combined power of Python and C to let you 
- write Python code that calls back and forth from and to C or C++ code natively at any point.
- easily tune readable Python code into plain C performance by adding static type declarations.
- use combined source code level debugging to find bugs in your Python, Cython and C code.
- interact efficiently with large data sets, e.g. using multi-dimensional NumPy arrays.
- quickly build your applications within the large, mature and widely used CPython ecosystem.
- integrate natively with existing code and data from legacy, low-level or high-performance libraries and applications.

# Przykład

In [1]:
def fib(n):
    if n<2:
        return n
    return fib(n-1)+fib(n-2)

In [2]:
fib(20)

6765

In [3]:
# fib(100) # it tooks a lot of time ....

In [4]:
%timeit fib(20)

100 loops, best of 3: 2.34 ms per loop


It means that the timer did the following:
- Run fib(20) one hundred times, store the total running time
- Run fib(20) one hundred times, store the total running time
- Run fib(20) one hundred times, store the total running time
- Get the smallest running time from the three runs, divide it by 100, and outputs the result as the best running time for fib(20)

## Compiling With Cython

In [5]:
!pip install Cython

Once installed, we load Cython in the notebook with the %load_ext magic

In [6]:
%load_ext Cython

We can then compile code in our notebook.  All we have to do is to put all the code we want to compile in one cell, including the required import statements, and start that cell with the cell magic `%%cython`:

In [7]:
%%cython
def fib_cython(n):
    if n<2:
        return n
    return fib_cython(n-1)+fib_cython(n-2)

In [8]:
%timeit fib_cython(20)

1000 loops, best of 3: 1.01 ms per loop


Wow, more than 2 times faster than the original Python code!

We can also try with static typing.  We declare the function with the keyword cpdef instead of def.  It allows us to type the parameters of the function with their corresponding C types.  Our code becomes.

In [9]:
%%cython
 
cpdef long fib_cython_type(long n):
    if n<2:
        return n
    return fib_cython_type(n-1)+fib_cython_type(n-2)

In [10]:
%timeit fib_cython_type(20)

10000 loops, best of 3: 37.9 µs per loop


| function  | helper | time  | 
|---|---|---|
| fib  | pure python | 2.32 ms |
| fib_cython  | pure cython | 1.02 ms |
| fib_cython_type  | c-type | 37 µs |

One can argue that static typing defeats the purpose of Python.  I kind of agree with that in general, and we will see later a way to avoid this without sacrificing performance.  But I don't think this is the issue here.  The Fibonacci function is meant to be called with integers.  What we lose with static typing is the arbitrary precision that Python provides.  In the case of Fibonacci, using the C type long limits the size of the input parameter because too large parameters would result in integer overflow. 

### It's not over yet

## Caching Computation

We can do better while keeping Python arbitrary precision.  The fib function repeats the same computation many times.  For instance, fib(20) will call fib(19) and fib(18).  In turn, fib(19) will call fib(18) and fib(17).  As a result fib(18) will be called twice.  A little analysis shows that fib(17) will be called 3 times, and fib(16) five times, etc. 

In [11]:
!pip install backports.functools_lru_cache

In [12]:
try:
    from functools import lru_cache as cache
except ImportError:
    from backports.functools_lru_cache import lru_cache as cache

In [13]:
@cache(maxsize=None)
def fib_cache(n):
    if n<2:
        return n
    return fib_cache(n-1)+fib_cache(n-2)

In [14]:
%timeit fib_cache(20)

The slowest run took 265.57 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 709 ns per loop


| function  | helper | time  | 
|---|---|---|
| fib  | pure python | 2.32 ms |
| fib_cython  | pure cython | 1.02 ms |
| fib_cython_type  | c-type | 37 µs |
| fib_cache  | pure python + cache | 709 ns |

## Iterative fib

In [15]:
def fib_seq(n):
    if n < 2:
        return n
    a,b = 1,0
    for i in range(n-1):
        a,b = a+b,a
    return a   

In [16]:
%timeit fib_seq(20)

The slowest run took 20.35 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 1.26 µs per loop


In [17]:
%%cython

def fib_seq_cython(n):
    if n < 2:
        return n
    a,b = 1,0
    for i in range(n-1):
        a,b = a+b,a
    return a    

cpdef long fib_seq_cython_type(long n):
    if n < 2:
        return n
    cdef long a,b
    a,b = 1,0
    for i in range(n-1):
        a,b = a+b,a
    return a

In [18]:
%timeit fib_seq_cython(20)
%timeit fib_seq_cython_type(20)

The slowest run took 6.45 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 597 ns per loop
The slowest run took 24.44 times longer than the fastest. This could mean that an intermediate result is being cached.
10000000 loops, best of 3: 52.5 ns per loop


| function  | helper | time  | 
|---|---|---|
| fib  | pure python | 2.32 ms |
| fib_cython  | cython | 1.02 ms |
| fib_cython_type  | cython | 37 µs |
| fib_cache  | pure python + cache | 709 ns |
| fib_seq  | pure python | 1.28 µs |
| fib_seq_cython  | cython | 624 ns |
| fib_seq_cython_type  |cython | 52.7 ns |

## Compiling With Numba

![alt text](images/numba_blue_icon_rgb.png "numba_blue_icon_rgb")

Let us use another tool called Numba.  It is a just in time (jit) compiler for a subset of Python.  It does not work yet on all of Python, but when it does work it can do marvels. 

In [19]:
from numba import jit

In [20]:
!pip install numba

In [21]:
@jit
def fib_seq_numba(n):
    if n < 2:
        return n
    a,b = 1,0
    for i in range(n-1):
        a,b = a+b,a
    return a  

In [22]:
%timeit fib_seq_numba(20)

The slowest run took 447678.60 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 210 ns per loop


| function  | helper | time  | 
|---|---|---|
| fib  | pure python | 2.32 ms |
| fib_cython  | cython | 1.02 ms |
| fib_cython_type  | cython | 37 µs |
| fib_cache  | pure python + cache | 709 ns |
| fib_seq  | pure python | 1.28 µs |
| fib_seq_cython  | cython | 624 ns |
| fib_seq_cython_type  |cython | 52.7 ns |
| fib_seq_numba  | numba | 211 ns |

## Przykład II

In [23]:
import random
def parse_int():
    for i in range(1,1000):
        n = random.randint(0,2**32-1)
        s = hex(n)
        if s[-1]=='L':
            s = s[0:-1]
        m = int(s,16)
        assert m == n

In [24]:
%timeit parse_int()

100 loops, best of 3: 2.24 ms per loop


In [25]:
%%cython
import random
def parse_int_cython():
    for i in range(1,1000):
        n = random.randint(0,2**32-1)
        s = hex(n)
        if s[-1]=='L':
            s = s[0:-1]
        m = int(s,16)
        assert m == n

In [26]:
@jit
def parse_int_numba():
    for i in range(1,1000):
        n = random.randint(0,2**32-1)
        s = hex(n)
        if s[-1]=='L':
            s = s[0:-1]
        m = int(s,16)
        assert m == n

In [27]:
%timeit parse_int_cython()
%timeit parse_int_numba()

100 loops, best of 3: 2.23 ms per loop
The slowest run took 68.13 times longer than the fastest. This could mean that an intermediate result is being cached.
100 loops, best of 3: 2.66 ms per loop


### ... did not work :(

#### We can use profiler to egzamine the code

In [28]:
%load_ext line_profiler

In [29]:
!pip install line_profiler

In [54]:
%lprun -s -f parse_int parse_int()

We see that most of the time is in generating the random numbers.  This was the intent of the benchmark...

## ... numpy?

In [31]:
import numpy as np
def parse_int1_numpy():
    for i in range(1,1000):
        n = np.random.randint(0,2**31-1)
        s = hex(n)
        m = int(s,16)
        assert m == n

In [32]:
%timeit parse_int1_numpy()

1000 loops, best of 3: 1.69 ms per loop


In [33]:
%%cython
import numpy as np
cimport cython

@cython.boundscheck(False)
# If set to False, Cython is free to assume that indexing operations ([]-operator) 
# in the code will not cause any IndexErrors to be raised.
@cython.wraparound(False)
# In Python arrays can be indexed relative to the end. For example A[-1] indexes 
# the last value of a list. In C negative indexing is not supported. If set to False, 
# Cython will not ensure that python indexing is not used.

cpdef parse_int_cython_numpy():
    cdef:
        int i,n,m
    for i in range(1,1000):
        n = np.random.randint(0,2**31-1)
        m = int(hex(n),16)
        assert m == n

https://github.com/cython/cython/wiki/enhancements-compilerdirectives

In [34]:
%timeit parse_int_cython_numpy()

1000 loops, best of 3: 1.47 ms per loop


### ... maybe the order of oparations?

In [35]:
def parse_int_vec():
    n = np.random.randint(2^31-1,size=1000)
    for i in range(1,1000):
        ni = n[i]
        s = hex(ni)
        if s[-1]=='L':
            s = s[0:-1]
        m = int(s,16)
        assert m == ni

In [36]:
%timeit parse_int_vec()

1000 loops, best of 3: 758 µs per loop


### ... numpy once again

In [37]:
vhex = np.vectorize(hex)
vint = np.vectorize(int)

def parse_int_numpy():
    n = np.random.randint(0,2**31-1,1000)
    s = vhex(n)
    m = vint(s,16)
    np.all(m == n)
    return s

In [38]:
%timeit parse_int_numpy()

1000 loops, best of 3: 449 µs per loop


### ... and the winner is ...

In [39]:
%%cython
import numpy as np
import cython

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef parse_int_vec_cython():
    cdef:
        int i,m
        int[:] n
    n = np.random.randint(0,2**31-1,1000)
    for i in range(1,1000):
        m = int(hex(n[i]),16)
        assert m == n[i]

In [40]:
%timeit parse_int_vec_cython()

1000 loops, best of 3: 268 µs per loop


## Przykład III

In [41]:
import numpy as np
A = np.random.randint(5, size=(100, 100), dtype=np.uint8)
B = np.random.randint(5, size=(100, 100), dtype=np.uint8)

In [42]:
import numpy as np

def my_add(a, b):
    dtype = a.dtype
    height = a.shape[0]
    width = a.shape[1]

    result = np.zeros((height, width), dtype=dtype)

    for y in range(height):
        for x in range(width):
            result[y,x] = a[y,x] + b[y,x]

    return result

In [43]:
%timeit my_add(A,B)

100 loops, best of 3: 3.99 ms per loop


In [44]:
%%cython
import numpy as np

def my_add_cython(a, b):
    dtype = a.dtype
    height = a.shape[0]
    width = a.shape[1]

    result = np.zeros((height, width), dtype=dtype)

    for y in range(height):
        for x in range(width):
            result[y,x] = a[y,x] + b[y,x]

    return result

In [45]:
%timeit my_add_cython(A,B)

100 loops, best of 3: 3.12 ms per loop


In [46]:
%%cython
import numpy as np
cimport numpy as np

DTYPE = np.uint8

def my_add_cython_with_types(np.ndarray a, np.ndarray b):
    
    cdef int height = a.shape[0]
    cdef int width = a.shape[1]

    cdef np.ndarray result = np.zeros((height, width), dtype=DTYPE)
    
    cdef int x, y
    for y in range(height):
        for x in range(width):
            result[y,x] = a[y,x] + b[y,x]

    return result

In [47]:
%timeit my_add_cython_with_types(A,B)

100 loops, best of 3: 3.11 ms per loop


In [48]:
%%cython
import numpy as np
cimport numpy as np

DTYPE = np.uint8
ctypedef np.uint8_t DTYPE_t

def my_add_cython_with_types_2(np.ndarray[DTYPE_t,ndim=2] a, np.ndarray[DTYPE_t,ndim=2] b):
    
    cdef int height = a.shape[0]
    cdef int width = a.shape[1]

    cdef np.ndarray[DTYPE_t, ndim=2] result = np.zeros((height, width), dtype=DTYPE)
    
    cdef int x, y
    for y in range(height):
        for x in range(width):
            result[y,x] = a[y,x] + b[y,x]

    return result

In [49]:
%timeit my_add_cython_with_types_2(A,B)

10000 loops, best of 3: 52.7 µs per loop


In [50]:
from numba.decorators import jit
import numpy as np

@jit('uint8[:,:](uint8[:,:], uint8[:,:])')
def my_add_numba(a, b):
    dtype = a.dtype
    height = a.shape[0]
    width = a.shape[1]

    result = np.zeros((height, width), dtype=dtype)

    for y in range(height):
        for x in range(width):
            result[y,x] = a[y,x] + b[y,x]

    return result

In [51]:
%timeit my_add_numba(A,B)

The slowest run took 44.37 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 8.66 µs per loop


## ... but the winner is ...

In [52]:
def my_add_numpy(a, b):
    return a+b

In [53]:
%timeit my_add_numpy(A,B)

The slowest run took 10.14 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 7.47 µs per loop


Sources:
- http://jakevdp.github.io/blog/2012/08/24/numba-vs-cython/
- https://www.ibm.com/developerworks/community/blogs/jfp/entry/Python_Meets_Julia_Micro_Performance?lang=en
- https://www.youtube.com/watch?v=SUf-ALvk3cU


For slideshow in jupyter
https://github.com/damianavila/RISE

https://www.youtube.com/watch?v=NfnMJMkhDoQ